Document comparison

# Travel planning requirements vs. generated day-to-day plan using available itinerary information

* Flight, Hotel, Airport parking, Car rental

* Desired activities and destinations


## Configure OpenAI Settings

In [1]:
import os
import openai
from dotenv import load_dotenv
# Set up Azure OpenAI
load_dotenv()

openai.api_type = "azure"

AZURE_OPENAI_API_VERSION = os.getenv("AAG_AZURE_OPENAI_API_VERSION")
openai.api_version = AZURE_OPENAI_API_VERSION

AZURE_OPENAI_API_KEY = os.getenv("AAG_AZURE_OPENAI_API_KEY").strip()
assert AZURE_OPENAI_API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = AZURE_OPENAI_API_KEY

AZURE_OPENAI_ENDPOINT = os.getenv("AAG_AZURE_OPENAI_ENDPOINT","").strip()
assert AZURE_OPENAI_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
openai.api_base = AZURE_OPENAI_ENDPOINT

# Deployment for Chat
# DEPLOYMENT_NAME_CHAT = os.getenv('DEPLOYMENT_NAME_CHAT')
DEPLOYMENT_NAME_CHAT = os.getenv('AAG_DEPLOYMENT_NAME_CHAT_16K')

# Deployment for embedding
DEPLOYMENT_NAME_EMBEDDING = os.getenv("AAG_DEPLOYMENT_NAME_EMBEDDING")
model: str = DEPLOYMENT_NAME_EMBEDDING

# Azure AI Search (Cognitive vector store)
vector_store_address: str = os.getenv("AAG_AZURE_SEARCH_SERVICE_ENDPOINT")  
vector_store_password: str = os.getenv("AAG_AZURE_SEARCH_ADMIN_KEY")
# index_name: str = "langchain-vector-arxiv-physics"

# Deployment for embedding
BING_SUBSCRIPTION_KEY = os.getenv("BING_SUBSCRIPTION_KEY")

## requirement

In [2]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader("./ask/trip_planning_requirement.txt", encoding="utf-8")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs_request = text_splitter.split_documents(documents)


#### Split documents into chunks

In [ ]:
docs_request

In [4]:
loader = TextLoader("./reports/trip_report.txt", encoding="utf-8")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs_report = text_splitter.split_documents(documents)

In [ ]:
docs_report

In [6]:
splitted_docs = docs_request + docs_report

In [ ]:
splitted_docs

### Create embeddings and vector store instances

### FAISS vector store

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Get Azure OpenAI embedding
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(
    deployment=model,
    model=model,
    chunk_size=1,   # this 'chunk_size' is misleading, it is really about 'input' text string, not the number of words or characters in the text.
    openai_api_base=AZURE_OPENAI_ENDPOINT,
    openai_api_type="azure",
    api_key=AZURE_OPENAI_API_KEY,
)

# Create the vector index
db = FAISS.from_documents(splitted_docs, embeddings)

In [8]:
# Set up the retriever we want to use, and then turn it into a retriever tool
faiss_db_retriever = db.as_retriever()

In [9]:
from langchain.agents.agent_toolkits import create_retriever_tool

# Tool define
faiss_retriever = create_retriever_tool(
    faiss_db_retriever,
    "Travel plan comparison",
    "This tool is help to compare the Ski trip planning requirement vs generated trip report",
)

In [10]:
from langchain.chat_models import AzureChatOpenAI
# Azure OpenAI model
llm = AzureChatOpenAI(
    azure_deployment="gpt-35-turbo-16k",
    temperature=0,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    openai_api_type="azure",
    api_key=AZURE_OPENAI_API_KEY,
)

In [11]:
from langchain.agents import initialize_agent, AgentType
# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
faiss_retriever_tools = [
    faiss_retriever,
]

# tools = load_tools(["bing-search"], llm=llm)
test_agent = initialize_agent(
    faiss_retriever_tools,
    llm,
    agent="zero-shot-react-description",
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5,
)


In [12]:
# Now let's test it out!
test_agent.run("Is trip report matching the trip planning requirements? If you rate it (0-no match, 9-best match), what is the score?")



> Entering new AgentExecutor chain...
I need to compare the trip report with the trip planning requirements to determine if they match. I should rate the match on a scale of 0 to 9.
Action: Travel plan comparison
Action Input: Trip report, Trip planning requirements
Observation: [Document(page_content='Source: Conversation with Bing, 12/1/2023\n(1) Planning a Ski Trip: The Complete Guide - TripSavvy. https://www.tripsavvy.com/planning-a-ski-trip-complete-guide-4782195.\n(2) What to Pack for a Ski Trip: A Complete Checklist - Travel. https://www.travelandleisure.com/what-to-pack-for-a-ski-trip-6823629.\n(3) How to rent a car for your ski trip - Rentalcars.com. https://www.rentalcars.com/en/guides/rental-basics/rent-ski-winter/.\n(4) How to Plan a Ski Trip in 5 Easy Steps | Ship Skis. https://bing.com/search?q=travel+steps+for+a+ski+trip.\n(5) The PERFECT Ski Trip Packing List (2023) - The World Pursuit. https://theworldpursuit.com/ski-trip-packing-list/.\n(6) Skiing in China: 4 Steps 

'The score for the match between the trip report and the trip planning requirements is 9.'

# Score is 7 out of 9

In [20]:
# Now let's test it out!
test_agent.run("Please compare trip_report.txt and  trip_planning_requirement.txt and highlight the difference")



> Entering new AgentExecutor chain...
I need to compare the contents of the two text files and identify any differences.
Action: Travel plan comparison
Action Input: trip_report.txt, trip_planning_requirement.txt
Observation: [Document(page_content="Here's a detailed plan including travel steps:\n\n**Before Departure:**\n1. **Book Your Flight**: Choose a flight that arrives at your destination in the morning or early afternoon, this will give you time to get to your accommodation and settle in¹.\n2. **Airport Parking**: Reserve your airport parking spot in advance to ensure availability and save money¹.\n3. **Pack Your Bags**: Pack your ski gear and clothes. Don't forget essentials like ski jackets, pants, gloves, goggles, and warm clothing².\n\n**Day of Travel:**\n4. **Check-In**: Arrive at the airport early to check-in for your flight¹.\n5. **Security Check**: Go through security check. Remember to follow all TSA guidelines¹.\n6. **Board Your Flight**: Board your flight and enjoy t

'The differences between the trip_report.txt and trip_planning_requirement.txt files are as follows:\n\n1. The trip_report.txt file provides a detailed day-to-day ski trip plan for a week-long Ski SLC trip, including information on departure and arrival airports, rental car pickup, hotel check-in, skiing at Solitude Mountain Resort, and dining options. The trip_planning_requirement.txt file does not contain this specific day-to-day plan.\n\n2. The trip_report.txt file includes contact information for the Fairfield Inn & Suites Salt Lake City South hotel, such as the address and phone number. This information is not present in the trip_planning_requirement.txt file.\n\n3. The trip_planning_requirement.txt file provides a general plan for a ski trip, including steps such as booking a flight, reserving airport parking, and packing ski gear. This information is not included in the trip_report.txt file.\n\nBased on these differences, it seems that the trip_report.txt file provides a more de

In [23]:
test_agent.run("Please compare trip_report and trip_planning_requirement, then highlight the difference")



> Entering new AgentExecutor chain...
I should use the Travel plan comparison tool to compare the trip_report and trip_planning_requirement.
Action: Travel plan comparison
Action Input: trip_report, trip_planning_requirement
Observation: [Document(page_content='Source: Conversation with Bing, 12/1/2023\n(1) Planning a Ski Trip: The Complete Guide - TripSavvy. https://www.tripsavvy.com/planning-a-ski-trip-complete-guide-4782195.\n(2) What to Pack for a Ski Trip: A Complete Checklist - Travel. https://www.travelandleisure.com/what-to-pack-for-a-ski-trip-6823629.\n(3) How to rent a car for your ski trip - Rentalcars.com. https://www.rentalcars.com/en/guides/rental-basics/rent-ski-winter/.\n(4) How to Plan a Ski Trip in 5 Easy Steps | Ship Skis. https://bing.com/search?q=travel+steps+for+a+ski+trip.\n(5) The PERFECT Ski Trip Packing List (2023) - The World Pursuit. https://theworldpursuit.com/ski-trip-packing-list/.\n(6) Skiing in China: 4 Steps to Plan a Ski Trip - China Highlights. htt

'The final answer is that the Travel plan comparison tool has compared the trip_report and trip_planning_requirement.'